<a href="https://colab.research.google.com/github/y-richie-y/no_eating_no_drinking/blob/master/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import time

th.manual_seed(1)

$p$ is the pooling size of each maxout layer. "We use a maxout pool size of 16"

In [0]:
# Pooling size of each maxout layer
MAXOUT_POOL_SIZE = 16

ENCODING_DIM = 200

DROPOUT = 0.3

document_sequence_size = 70
question_sequence_size = 80

# Hidden state dimension of the LSTM
HIDDEN_SIZE = 200

In [0]:
### 2.1 DOCUMENT AND QUESTION ENCODER

# The encoder LSTM.
class Encoder(nn.Module):
  def __init__(self, doc_word_vecs, question_word_vecs, hidden_dim, dropout, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.hidden_dim = hidden_dim

    # Dimensionality of word vectors.
    self.word_vec_dim = doc_word_vecs.size()[1]
    assert(self.word_vec_dim == question_word_vecs.size()[1])

    # Dimension of the hidden state and cell state (they're equal) of the LSTM
    self.lstm = nn.LSTM(self.word_vec_dim, hidden_dim, 1, batch_first=True, bidirectional=False, dropout=dropout)

  def forward(self, x, hidden):
    return self.lstm(x, hidden)

  def generate_initial_hidden_state(self):
    return (th.zeros(self.hidden_dim).view(1,batch_size,self.hidden_dim),
            th.zeros(self.hidden_dim).view(1,batch_size,self.hidden_dim))

In [0]:
class BiLSTMEncoder(nn.Module):

    def __init__(self, hidden_dim, document_sequence_size, batch_size, dropout=DROPOUT, use_gpu = False):
        super(BiLSTMEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.document_sequence_size=document_sequence_size
        self.dropout = dropout
        self.lstm = nn.LSTM(3 * hidden_dim, hidden_dim, 1, batch_first=True, bidirectional=True, dropout=dropout)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        # TODO: Is initialisation zeros or randn? 
        return (th.zeros(2, self.batch_size, self.hidden_dim),
              th.zeros(2, self.batch_size,self.hidden_dim))
            #return (Variable(th.zeros(num_layers, self.batch_size, self.hidden_dim).to(device)),
            #Variable(th.zeros(num_layers, self.batch_size, self.hidden_dim).to(device)))
        

    def forward(self, input_BiLSTM):
        print('Input shape: ', input_BiLSTM.shape[0], " ", self.document_sequence_size)
        lstm_out, self.hidden = self.lstm(input_BiLSTM.reshape(input_BiLSTM.shape[0], self.document_sequence_size, -1), self.hidden)
        
        
        return lstm_out

In [0]:
class CoattentionModel(nn.Module):
    def __init__(self, hidden_dim, dropout_ratio):
        super(CoattentionModel, self).__init__()
        self.hidden_dim = hidden_dim

        #self.encoder = Encoder(hidden_dim, emb_matrix, dropout_ratio) #Kuba

        #self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        #self.fusion_bilstm = FusionBiLSTM(hidden_dim, dropout_ratio)
        #self.decoder = DynamicDecoder(hidden_dim, maxout_pool_size, max_dec_steps, dropout_ratio)
        #self.dropout = nn.Dropout(p=dropout_ratio)

    def forward(self, D, Q):
        #Q = self.encoder(q_seq, q_mask) # b x n + 1 x l
        #D = self.encoder(d_seq, d_mask)  # B x m + 1 x l
        #D=torch.randn(B,hidden_dim, document_sequence_size)#replace with what Kuba wrote #replace size by n+1
        #Q=torch.randn(B,hidden_dim, question_sequence_size) #replace with what kuba wrote #replace size by m+1 #B x n + 1 x l

        #co attention
        
        D_T = th.transpose(D, 1, 2) #B x l x m + 1 
        L = th.bmm(D_T, Q) # L = B x m + 1 x n + 1
        AQ = F.softmax(L, dim=1)
        AD_T = F.softmax(L,dim=2)
        AD = th.transpose(AD_T, 1, 2) # B x n + 1 x m + 1

        CQ = th.bmm(D,AQ) #R l×(n+1)
        CD = th.bmm(th.cat((Q,CQ),1),AD) # B x 2l x m + 1
        C_D_t = th.transpose(CD, 1, 2)  # B x m + 1 x 2l

        #fusion BiLSTM
        
        input_BiLSTM=th.transpose(th.cat((D,CD), 1),1,2)
        print(input_BiLSTM.shape)

        model = BiLSTMEncoder(hidden_dim, document_sequence_size, B)
        return model.forward(input_BiLSTM)


In [0]:
class HighwayMaxoutNetwork(nn.Module):
  def __init__(self, maxout_pool_size = MAXOUT_POOL_SIZE, encoding_dim = ENCODING_DIM): 
    super(HighwayMaxoutNetwork, self).__init__()

    self.encoding_dim = encoding_dim
    self.maxout_pool_size = MAXOUT_POOL_SIZE

    # Don't apply dropout to biases
    self.dropout = nn.Dropout(p=DROPOUT)

    # W_D := Weights of MLP applied to the coattention encodings of
    # the start/end positions, and the current LSTM hidden state (h_i)
    # (nn.Linear is an affine transformation y = Wx + b)

    # There are 5 * encoding_dim incoming features (u_si-1, u_ei-1, h_i) 
    # which are vectors containing (2l, 2l, l) elements respectively
    # There's l outgoing features (i.e. r)
    # There's no bias for this MLP
    
    # (From OpenReview) random initialisation is used for W's and b's
    self.W_D = self.dropout(nn.Parameter(th.randn(self.encoding_dim, 5 * self.encoding_dim)))

    # 1st Maxout layer
    self.W_1 = self.dropout(nn.Parameter(th.randn(self.maxout_pool_size, self.encoding_dim, 3 * self.encoding_dim)))
    self.b_1 = nn.Parameter(th.randn(self.maxout_pool_size, self.encoding_dim))

    # 2nd maxout layer
    self.W_2 = self.dropout(nn.Parameter(th.randn(self.maxout_pool_size, self.encoding_dim, self.encoding_dim)))
    self.b_2 = nn.Parameter(th.randn(self.maxout_pool_size, self.encoding_dim))

    # 3rd maxout layer
    self.W_3 = self.dropout(nn.Parameter(th.randn(self.maxout_pool_size, 1, 2 * self.encoding_dim)))
    self.b_3 = nn.Parameter(th.randn(self.maxout_pool_size, 1))

  def forward(self, u_t, h_i, u_si_m_1, u_ei_m_1):

    assert(u_t.size()[0] == 2 * self.encoding_dim) 
    assert(u_t.size()[1] == 1) 
    assert(h_i.size()[0] == self.encoding_dim)
    assert(h_i.size()[1] == 1)
    assert(u_si_m_1.size()[0] == 2 * self.encoding_dim)
    assert(u_si_m_1.size()[1] == 1)
    assert(u_ei_m_1.size()[0] == 2 * self.encoding_dim)
    assert(u_ei_m_1.size()[1] == 1)

    # r := output of MLP 
    r = th.tanh(self.W_D.mm(th.cat((h_i, u_si_m_1, u_ei_m_1), 0)))

    # m_t_1 := output of 1st maxout layer
    m_t_1_beforemaxpool = th.mm(
        self.W_1.view(self.maxout_pool_size * self.encoding_dim, 
                        3 * self.encoding_dim), 
                        th.cat((u_t, r), 0)
    ).view(
        self.maxout_pool_size, 
        self.encoding_dim, 
        1
    ).squeeze() + self.b_1

    # The max operation in Eq.9-12 computes the maximum value over the 
    # first dimension of a tensor
    m_t_1 = th.Tensor.max(m_t_1_beforemaxpool, dim=0).values.unsqueeze(dim=1)

    m_t_2_beforemaxpool = th.mm(
        self.W_2.view(self.maxout_pool_size * self.encoding_dim, 
                        self.encoding_dim), 
                        m_t_1
    ).view(
        self.maxout_pool_size, 
        self.encoding_dim, 
        1
    ).squeeze() + self.b_2
    m_t_2 = th.Tensor.max(m_t_2_beforemaxpool, dim=0).values.unsqueeze(dim=1)

    # HMN output
    output_beforemaxpool = th.mm(
        self.W_3.view(
            self.maxout_pool_size, 
            2 * self.encoding_dim
        ), 
        # Highway connection
        th.cat((m_t_1, m_t_2), 0)
    ) + self.b_3
    
    output = th.Tensor.max(output_beforemaxpool, dim=0).values
    return output


In [0]:
class DynamicPointerDecoder(nn.Module):
  def __init__(self):
    super(DynamicPointerDecoder, self).__init__()
    self.hmn_alpha = HighwayMaxoutNetwork()
    self.hmn_beta = HighwayMaxoutNetwork()
    self.lstm = nn.LSTM(4*HIDDEN_SIZE, HIDDEN_SIZE, 1, bidirectional=False, dropout=DROPOUT)


  def forward(self, U, max_iter):

    # TODO: Value to choose for max_iter (600?)
    # Initialise h_0, s_i_0, e_i_0 (TODO can change)
    s = 0
    e = 0

    # initialize the hidden and cell states 
    # hidden = (h, c)

    doc_length = U.size()[1]
    hidden = (th.randn(1,1,HIDDEN_SIZE), th.randn(1,1,HIDDEN_SIZE))
    
    # "The iterative procedure halts when both the estimate of the start position 
    # and the estimate of the end position no longer change, 
    # or when a maximum number of iterations is reached"

    # We build up the losses here (the iteration being the first dimension)
    alphas = th.tensor([]).view(0, doc_length)
    betas = th.tensor([]).view(0, doc_length)

    # TODO: make it run only until convergence (or maxiter)
    for _ in range(max_iter):
      # call LSTM to update h_i

      # Step through the sequence one element at a time.
      # after each step, hidden contains the hidden state.
      u_concatenated = th.cat((U[:,s], U[:,e]), dim=0).view(-1, 1)
      #print("u_concatenated.size()", u_concatenated.size())

      out, hidden = self.lstm(u_concatenated.view(1, 1, -1), hidden)
      h, _ = hidden

      # Call HMN to update s_i, e_i
      alpha = th.tensor([]).view(0, 1)
      beta = th.tensor([]).view(0, 1)

      for t in range(doc_length):
        t_hmn_alpha = self.hmn_alpha(U[:, t].view(-1, 1), h.view(-1, 1), U[:,s].view(-1, 1), U[:,e].view(-1, 1))
        t_hmn_beta = self.hmn_beta(U[:, t].view(-1, 1), h.view(-1, 1), U[:,s].view(-1, 1), U[:,e].view(-1, 1))
        alpha = th.cat((alpha, t_hmn_alpha.view(1, 1)), dim=0)
        beta = th.cat((beta, t_hmn_beta.view(1, 1)), dim=0)

      # Leaving out dim=0 changes behaviour
      _, s = th.max(alpha, dim=0)
      _, e = th.max(beta, dim=0)
      
      alphas = th.cat((alphas, alpha.view(1, -1)), dim=0)
      betas = th.cat((betas, beta.view(1, -1)), dim=0)

    return (alphas, betas, s, e)

In [0]:


dpd = DynamicPointerDecoder()
max_iter = 10
U = th.ones(2 * HIDDEN_SIZE, 50)
alphas, betas, s, e = dpd.forward(U, max_iter)
loss = th.mean(th.mean(alphas, dim=0)) + th.mean(th.mean(betas, dim=0))
loss.backward()
print(loss)
#for param in dpd.parameters():
#  print(param.grad)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor(114774.6719, grad_fn=<AddBackward0>)


In [0]:
# The full model.
class DCNModel(nn.Module):
  def __init__(self, doc_word_vecs, question_word_vecs, encoder_h_dim, encoder_dropout):
    super(DCNModel, self).__init__()
    self.encoder_h_dim = encoder_h_dim
    self.encoder = Encoder(doc_word_vecs, question_word_vecs, encoder_h_dim, encoder_dropout)
    print("Encoder shape: ",self.encoder)
    self.encoder_sentinel = nn.Parameter(th.randn(encoder_h_dim)) # the sentinel is a trainable parameter of the network
    self.WQ = nn.Linear(encoder_h_dim, encoder_h_dim)
    self.dyn_ptr_dec = DynamicPointerDecoder() 
    self.coattention = CoattentionModel(encoder_h_dim, encoder_dropout)

  def forward(self, doc_word_vecs, question_word_vecs):
    
    # TODO: how should we initialise the hidden state of the LSTM? For now:
    hidden = self.encoder.generate_initial_hidden_state()
    for i in doc_word_vecs:
        outp, hidden = self.encoder.lstm(i.view(1, 1, -1), hidden)

    D = th.cat([outp.view(-1), self.encoder_sentinel], 0)  # append sentinel word vector

    # TODO: Make sure we should indeed reinit hidden state before encoding the q.
    hidden = self.encoder.generate_initial_hidden_state()
    for i in question_word_vecs:
        outp, hidden = self.encoder.lstm(i.view(1, 1, -1), hidden)

    Qprime = th.cat([outp.view(-1), self.encoder_sentinel], 0)  # append sentinel word vector
    Q = th.tanh(self.WQ(Qprime.view(-1, self.encoder_h_dim))).view(Qprime.size())

    U = self.coattention.forward(D.unsqueeze(dim=0),Q.unsqueeze(dim=0))
    
    # TODO: Replace with the true start/end positions for the current batch of questions
    x,y = th.randint(0, doc_word_vecs.size()[0], (2,))
    x, y = min(x,y), max(x,y)
    
    # TODO: Replace with actual U
    max_iters = 10
    doc_words = doc_word_vecs.size()[0]

    criterion = nn.CrossEntropyLoss()    

    # Run the dynamic pointer decoder, accumulate the 
    # distributions over the start positions (alphas)
    # and end positions (betas) at each iteration
    # as well as the final start/end indices 
    
    alphas, betas, start, end = self.dyn_ptr_dec.forward(U, max_iters)

    # Accumulator for the losses incurred across 
    # iterations of the dynamic pointing decoder
    loss = th.FloatTensor([0])
    for it in range(max_iters):
      loss += criterion(alphas[it].view(1, -1), Variable(th.LongTensor([x])))
      loss += criterion(betas[it].view(1, -1), Variable(th.LongTensor([x])))
 
    return loss, start, end


In [0]:
# TEST
# DCNModel Test.
doc = th.randn(30, ENCODING_DIM)
que = th.randn(5, ENCODING_DIM)

# Run model.
model = DCNModel(doc, que, ENCODING_DIM, DROPOUT)
loss, s, e = model.forward(doc, que)
print("loss: ", loss, ", s:", s, ", e:", e)
model.zero_grad()
loss.backward()

print("%d/%d parameters are not None." % (len([param for param in model.parameters() if param is not None]), len(list(model.parameters()))))

Encoder shape:  Encoder(
  (lstm): LSTM(200, 200, dropout=0.3)
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


IndexError: ignored

In [0]:
doc.shape

In [0]:
# Optimiser 

doc = th.ones(30, ENCODING_DIM)
que = th.ones(5, ENCODING_DIM)
model = DCNModel(doc, que, ENCODING_DIM, 0.3)

# TODO: hyperparameters?
optimizer = optim.Adam(model.parameters())
n_iters = 1000 

# TODO: batching? 
for iter in range(n_iters):
  optimizer.zero_grad()
  loss, _, _ = model(doc, que)
  loss.backward()
  optimizer.step()

In [0]:
# TEST
# HMN test

hmn = HighwayMaxoutNetwork()
u_t = th.ones(2 * ENCODING_DIM, 1)
h_i = th.ones(ENCODING_DIM, 1)
u_si_m_1, u_ei_m_1 = th.ones(2 * ENCODING_DIM, 1), th.ones(2 * ENCODING_DIM, 1)
output = hmn.forward(u_t, h_i, u_si_m_1, u_ei_m_1)
#print(output)
output.backward()

#for param in hmn.parameters(): 
#  print(param.grad)

In [0]:
# TEST
# Dynamic pointer decoder test
dpe = DynamicPointerDecoder()
alphas, betas, _, _ = dpe.forward(th.randn(2 * HIDDEN_SIZE, 50), 10)
dpe.zero_grad()